Importanto Libs

In [ ]:
import pandas as pd

Importando dados

In [ ]:
def ler_arquivo(nome_arquivo):
    try: 
        return pd.read_excel(f'bases/{nome_arquivo}.xlsx')
    except FileNotFoundError as e: 
        return f'Erro: {e}'

In [ ]:
df = ler_arquivo('dados de entregas')

SLA

In [ ]:
# Total de entregas
df_total_entregas = df.loc[ df['status'] == 'ENTREGUE' ]

# Total de entregas sem atraso
sla_ok = df_total_entregas.loc[ df_total_entregas['dias_atraso'] == 0].shape[0]

# Percentual SLA
sla_percentual = (sla_ok / df_total_entregas.shape[0]) * 100

f'{round(sla_percentual, 1)}% das entregas foram feitas dentro do prazo.' 

KPI - Entregas atrasadas

In [ ]:

percentual_atraso = (
    df_total_entregas.loc[ df_total_entregas['dias_atraso']  > 0].shape[0] /
    df_total_entregas.shape[0]
) * 100

round(percentual_atraso, 1)

SLA - Identificando onde as operações funcionam melhor

In [ ]:
sla_filial = df_total_entregas.groupby('filial')['dias_atraso'].mean()

round(sla_filial, 1)

In [ ]:
sla_filial = df_total_entregas.groupby('filial')['dias_atraso'].apply(
    lambda grupo: (grupo == 0).mean() * 100
).sort_values(ascending=False)

a = sla_filial.reset_index(name='% Entregas no Prazo')
round(a, 1)

In [ ]:
# Validando palavra chave sem trabalhar com dataframe (tabela)

desc = 'python é bom demais, a linguagem mais tranquila que existe'.lower()

palavras = desc.split()
try:
    palavras.index('Python')
except ValueError as e:
    print(e)

Padronização da descrição

In [25]:
df['descricao_entrega'] = (
    df['descricao_entrega']
    .str.lower()
    .str.strip()
)

Separar as entregas atrasadas

In [26]:
df_atrasos = df.loc[ df['dias_atraso'] > 0 ].copy()

Categorizando as causas dos atrasos

In [27]:
df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('chuva|temporal|tempestade', na=False),
    "causa_atraso"
] = "Clima"

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('operacional|rota', na=False),
    "causa_atraso"
] = "Problema Operacional"

df_atrasos.loc[
    df_atrasos['descricao_entrega'].str.contains('cliente', na=False),
    "causa_atraso"
] = "Cliente Ausente"

df_atrasos['causa_atraso'] = df_atrasos['causa_atraso'].fillna("Outros")

df_atrasos['causa_atraso'].head(10)

0              Outros
2     Cliente Ausente
3     Cliente Ausente
5               Clima
6              Outros
7              Outros
9              Outros
12             Outros
13             Outros
14             Outros
Name: causa_atraso, dtype: str

Principais causas de atrasos

In [28]:
round(df_atrasos['causa_atraso'].value_counts(normalize=True) * 100, 1).reset_index(name='%')

,causa_atraso,%
0,Outros,63.0
1,Cliente Ausente,12.7
2,Clima,12.7
3,Problema Operacional,11.7


Atrasos por filiais

In [ ]:
causas_por_filiais = df_atrasos            \
    .groupby(['filial', 'causa_atraso'])   \
    .size()                                \
    .reset_index(name='Qtd')               \
    .sort_values(['filial', 'Qtd'], ascending=[True, False])

total_por_filial = (
    causas_por_filiais.groupby('filial')['Qtd'].transform('sum')
)

causas_por_filiais['Percentual'] = (
    causas_por_filiais['Qtd'] / total_por_filial
) * 100

causas_por_filiais

,filial,causa_atraso,Qtd,Percentual
2,MG,Outros,350,60.553633
1,MG,Clima,84,14.532872
0,MG,Cliente Ausente,83,14.359862
3,MG,Problema Operacional,61,10.553633
6,PR,Outros,400,63.091483
5,PR,Clima,86,13.564669
7,PR,Problema Operacional,79,12.460568
4,PR,Cliente Ausente,69,10.883281
10,RJ,Outros,361,61.709402
8,RJ,Cliente Ausente,79,13.504274


Principal motivo de atraso de cada filial